# Homework 8 Question 3:  The Queens problem.
You are given a standard 8×8 chess board. The following problems
involve placing queens on the board such that certain constraints are satisfied. For each of the following
problems, model the optimization task as an integer program, solve it, and show what an optimal
placement of queens on the board looks like.

__a) Find a way to place 8 queens on the board so that no two queens threaten each other. We say
that two queens threaten each other if they occupy the same row, column, or diagonal. Show
what this placement looks like.__

In [1]:
# helper function to print a chess board
function printChessBoard(arr)
  u = 0
  println("+---+---+---+---+---+---+---+---+")
    for r in 1:8
        for c in 1:8
            if arr[r,c] == 1
                print("| ",round(Int,arr[r,c])," ")
            else
                print("|  "," ")
            end
        end
        println("|")
        println("+---+---+---+---+---+---+---+---+")
    end
end
;

In [2]:
# Helper function to generate neihbours
# Operations allowed to get neighbor of a cell.
# First column element is for row and second column for column operation
ops = [ 1  1;
       -1  1;
        1 -1;
       -1 -1]

function get_diag_nbr(i, j, rows, columns)
    retval = []
    for o in 1:length(ops[:,1])
        for mul in 1:8
            if i + ops[o,1]*mul <= rows && i + ops[o,1]*mul >= 1
                if j + ops[o,2]*mul <= columns && j + ops[o,2]*mul >= 1
                    push!(retval,(i + ops[o,1]*mul, j + ops[o,2]*mul))
                end
            end
        end
    end
    return retval
end
# Helper function to generate neihbours
# Operations allowed to get neighbor of a cell.
# First column element is for row and second column for column operation
all_ops = [ 1  1;
       -1  1;
        1 -1;
       -1 -1;
        0  1;
        0 -1;
        1  0;
       -1  0]

function get_all_nbr(i, j, rows, columns)
    retval = []
    for o in 1:length(all_ops[:,1])
        for mul in 1:8
            if i + all_ops[o,1]*mul <= rows && i + all_ops[o,1]*mul >= 1
                if j + all_ops[o,2]*mul <= columns && j + all_ops[o,2]*mul >= 1
                    push!(retval,(i + all_ops[o,1]*mul, j + all_ops[o,2]*mul))
                end
            end
        end
    end
    return retval
end;

In [3]:
println("Diagonal Neighbours for (4,4) as example")
[println("(",i,",",j,"): ",get_diag_nbr(i,j, 8,8)) for i in 4 for j in 4]
println("All Neighbours for (4,4) as example")
[println("(",i,",",j,"): ",get_all_nbr(i,j, 8,8)) for i in 4 for j in 4];;

Diagonal Neighbours for (4,4) as example
(4,4): Any[(5,5),(6,6),(7,7),(8,8),(3,5),(2,6),(1,7),(5,3),(6,2),(7,1),(3,3),(2,2),(1,1)]
All Neighbours for (4,4) as example
(4,4): Any[(5,5),(6,6),(7,7),(8,8),(3,5),(2,6),(1,7),(5,3),(6,2),(7,1),(3,3),(2,2),(1,1),(4,5),(4,6),(4,7),(4,8),(4,3),(4,2),(4,1),(5,4),(6,4),(7,4),(8,4),(3,4),(2,4),(1,4)]


In [4]:
using JuMP, Cbc

m = Model(solver = CbcSolver())

@variable(m, x[1:8,1:8], Bin)

# exactly one of each number per row
@constraint(m, rowC[i=1:8], sum(x[i,j] for j in 1:8) == 1)

# exactly one of each number per column
@constraint(m, colC[j=1:8], sum(x[i,j] for i in 1:8) == 1)

# if the sum of queens in the diagonals is 1 then this cell cannot contain queen
@constraint(m, diagC[i=1:8,j=1:8], sum(x[i_n,j_n] for (i_n,j_n) in get_diag_nbr(i,j,8,8)) - 1 
                                    <= 1 - x[i,j] - x[i,j]);

In [5]:
status = solve(m)
println("Status: ", status)
board = getvalue(x)
printChessBoard(board);

Status: Optimal
+---+---+---+---+---+---+---+---+
|   |   |   |   |   | 1 |   |   |
+---+---+---+---+---+---+---+---+
|   |   | 1 |   |   |   |   |   |
+---+---+---+---+---+---+---+---+
|   |   |   |   |   |   | 1 |   |
+---+---+---+---+---+---+---+---+
|   | 1 |   |   |   |   |   |   |
+---+---+---+---+---+---+---+---+
|   |   |   |   |   |   |   | 1 |
+---+---+---+---+---+---+---+---+
|   |   |   |   | 1 |   |   |   |
+---+---+---+---+---+---+---+---+
| 1 |   |   |   |   |   |   |   |
+---+---+---+---+---+---+---+---+
|   |   |   | 1 |   |   |   |   |
+---+---+---+---+---+---+---+---+


__b) Repeat part (a) but this time find a placement of the 8 queens that has point symmetry. In other
words, find a placement that looks the same if you rotate the board 180◦.__

In [6]:
m = Model(solver = CbcSolver())

@variable(m, x[1:8,1:8], Bin)

# exactly one of each number per row
@constraint(m, rowC[i=1:8], sum(x[i,j] for j in 1:8) == 1)

# exactly one of each number per column
@constraint(m, colC[j=1:8], sum(x[i,j] for i in 1:8) == 1)

# if the sum of queens in the diagonals is 1 then this cell cannot contain queen
@constraint(m, diagC[i=1:8,j=1:8], sum(x[i_n,j_n] for (i_n,j_n) in get_diag_nbr(i,j,8,8)) - 1 
                                    <= 1 - x[i,j] - x[i,j])

# symmetry constraint
@constraint(m, symC[i=1:8,j=1:8], x[i,j] == x[8-i+1,8-j+1]);

In [7]:
status = solve(m)
println("Status: ", status)
board = getvalue(x)
printChessBoard(board);

Status: Optimal
+---+---+---+---+---+---+---+---+
|   |   |   |   | 1 |   |   |   |
+---+---+---+---+---+---+---+---+
|   |   | 1 |   |   |   |   |   |
+---+---+---+---+---+---+---+---+
| 1 |   |   |   |   |   |   |   |
+---+---+---+---+---+---+---+---+
|   |   |   |   |   |   | 1 |   |
+---+---+---+---+---+---+---+---+
|   | 1 |   |   |   |   |   |   |
+---+---+---+---+---+---+---+---+
|   |   |   |   |   |   |   | 1 |
+---+---+---+---+---+---+---+---+
|   |   |   |   |   | 1 |   |   |
+---+---+---+---+---+---+---+---+
|   |   |   | 1 |   |   |   |   |
+---+---+---+---+---+---+---+---+


__c) What is the smallest number of queens that we can place on the board so that each empty cell is
threatened by at least one queen? Show a possible optimal placement.__

In [8]:
m = Model(solver = CbcSolver())

@variable(m, x[1:8,1:8], Bin)
# exactly one of each number per row
@constraint(m, rowC[i=1:8], sum(x[i,j] for j in 1:8) <= 1)

# exactly one of each number per column
@constraint(m, colC[j=1:8], sum(x[i,j] for i in 1:8) <= 1)

# diagonal constraint
@constraint(m, diagC[i=1:8,j=1:8], sum(x[i_n,j_n] for (i_n,j_n) in get_diag_nbr(i,j,8,8)) - 1 
                                    <= 1 - x[i,j] - x[i,j])
# attacking each cell at least once
@constraint(m, attackC[i=1:8,j=1:8], x[i,j] + sum(x[i_n,j_n] for (i_n,j_n) in get_all_nbr(i,j,8,8)) >= 1)
@objective(m, Min, sum(x))
;

In [9]:
status = solve(m)
println("Status: ", status)
println("Minimum number of queens required: ", getobjectivevalue(m))
board = getvalue(x)
printChessBoard(board);

Status: Optimal
Minimum number of queens required: 5.0
+---+---+---+---+---+---+---+---+
|   |   |   |   |   |   |   | 1 |
+---+---+---+---+---+---+---+---+
|   |   |   |   |   |   |   |   |
+---+---+---+---+---+---+---+---+
|   |   |   |   | 1 |   |   |   |
+---+---+---+---+---+---+---+---+
| 1 |   |   |   |   |   |   |   |
+---+---+---+---+---+---+---+---+
|   |   |   |   |   |   |   |   |
+---+---+---+---+---+---+---+---+
|   |   |   |   |   |   |   |   |
+---+---+---+---+---+---+---+---+
|   |   |   |   |   | 1 |   |   |
+---+---+---+---+---+---+---+---+
|   | 1 |   |   |   |   |   |   |
+---+---+---+---+---+---+---+---+


__d) Repeat part (c) but this time find a placement of the queens that also has point symmetry. Does
the minimum number of queens required change? Show a possible optimal placement.__

In [10]:
m = Model(solver = CbcSolver())

@variable(m, x[1:8,1:8], Bin)
# atmost one of each number per row
@constraint(m, rowC[i=1:8], sum(x[i,j] for j in 1:8) <= 1)

# atmost one of each number per column
@constraint(m, colC[j=1:8], sum(x[i,j] for i in 1:8) <= 1)
# diagonal constraint
@constraint(m, diagC[i=1:8,j=1:8], sum(x[i_n,j_n] for (i_n,j_n) in get_diag_nbr(i,j,8,8)) - 1 
                                    <= 1 - x[i,j] - x[i,j])
# attacking constraint
@constraint(m, attackC[i=1:8,j=1:8], x[i,j] + sum(x[i_n,j_n] 
                                            for (i_n,j_n) in get_all_nbr(i,j,8,8)) >= 1)
# symmetry constraint
@constraint(m, symC[i=1:8,j=1:8], x[i,j] == x[8-i+1,8-j+1])
@objective(m, Min, sum(x))
;

In [11]:
status = solve(m)
println("Status: ", status)
println("Minimum number of queens required: ", getobjectivevalue(m))
board = getvalue(x)
printChessBoard(board);

Status: Optimal
Minimum number of queens required: 6.0
+---+---+---+---+---+---+---+---+
|   |   |   |   |   |   | 1 |   |
+---+---+---+---+---+---+---+---+
|   |   | 1 |   |   |   |   |   |
+---+---+---+---+---+---+---+---+
|   |   |   |   |   |   |   |   |
+---+---+---+---+---+---+---+---+
|   |   |   |   |   |   |   | 1 |
+---+---+---+---+---+---+---+---+
| 1 |   |   |   |   |   |   |   |
+---+---+---+---+---+---+---+---+
|   |   |   |   |   |   |   |   |
+---+---+---+---+---+---+---+---+
|   |   |   |   |   | 1 |   |   |
+---+---+---+---+---+---+---+---+
|   | 1 |   |   |   |   |   |   |
+---+---+---+---+---+---+---+---+
